In [64]:
import loader
import random
cities, dist = loader.init("./data/uk12_code.txt","./data/uk12_dist.txt")
print("Number of cities is ", len(cities))


Number of cities is  12


In [65]:
# Calculate the fitness of a path (the sum of the distances between all cities)
def fitness(visitorder):
        return sum([dist[pair] for pair in zip(visitorder,visitorder[1:])]) + dist[(visitorder[0],visitorder[-1])]

In [66]:
# Generate a starting population 0 
def starting_population(size):
    population = []
    while len(population)<size:
        perm = random.sample(cities,len(cities))
        if perm not in population:
            population.append(perm)
    return population

In [67]:
# Crossover two parents
def crossover(p1, p2):
    # how much to cut off from parent
    cutoff = random.randint((1/3)*len(cities),(2/3)*len(cities))

    o1 = p1[:-cutoff]
    return o1 + [x for x in p2 if x not in o1] # filter the cities that have already been visited in o1

In [68]:
def next_city(p, city):
    # finds next visited city after 'city'
    current_ind = p.index(city)
    return (city,p[0]) if current_ind == len(p)-1 else (city,p[current_ind+1])
    
def heuristic_crossover(p1,p2):
    # For a pair of parents, pick a random city for the start. 
    # 2.    Choose the shortest edge (that is represented in the parents) leading from the current city which does not lead to a cycle. 
    # If two edges lead to a cycle, choose a random city that continues the tour. 
    # 3.    If the tour is completed, stop; otherwise go to step 2.
    offspring = []
    current_city = cities[random.randint(0,len(cities)-1)] # choose random start city
    offspring.append(current_city) # append it to child
    # find next city for each parent and take the minimum
    while (len(offspring) < len(p1)):
        pair1  = next_city(p1,current_city)
        pair2 = next_city(p2,current_city)
        cycle1 = pair1[1] in offspring
        cycle2 = pair2[1] in offspring

        if cycle1 and cycle2:
            offspring.append([city for city in cities if city not in offspring][0])
        elif cycle1:
            offspring.append(pair2[1])
        elif cycle2:
            offspring.append(pair1[1])
        else:
            offspring.append(min(pair1, pair2,key=lambda x: dist[x])[1])
        
        current_city = offspring[-1]
    return offspring

In [69]:
# Mutate an individual by re-inserting a random city at a random position
def mutate(p):
    o = p.copy() # copy the parent
    selected_city = random.choice(o) # random choice of city
    o.remove(selected_city) # remove that city
    o.insert(random.randint(0, len(o)), selected_city) # re-insert it at random position
    return o


In [70]:
# Randomly decided whether to mutate or crossover
from scipy.stats import bernoulli 
def mutate_or_crossover():
    return bernoulli.rvs(0.8) # 0.8 probability to choose crossover
    # 1 - crossover; 0 - mutation

In [71]:
# Create n offsprings from parents
def create_offsprings(parents, n):
    offsprings = []

    for i in range(0,n):
        op = mutate_or_crossover() # decide whether offspring i will be a crossover or a mutant
        if op == 0: 
            # if chosen operation is mutate, randomly choose parent from parents then mutate it
            offsprings.append(mutate(random.choice(parents)))
        else:
            # chosen operation is crossover
            [p1, p2] = random.sample(parents,k=2) # list
            offsprings.append(heuristic_crossover(p1,p2))
            
    return offsprings

In [72]:
# Select from the generation (size) individuals 
def selection(gen, size):
    p = [fitness(i) for i in gen]
    fitsum = sum(p)
    p = [pi/fitsum for pi in p] # probability of being selected

    return random.choices(gen,k=size,weights=p) # choose (size) individuals

In [73]:
# Retain k of the generation's candidates with highest fitness
def retain_best(gen, k):
    return sorted(gen,key=lambda x: fitness(x))[:k]

In [74]:
# Create the new generation by selecting some of the parents to create offsprings from, but also keep a few of the best parent individuals
def new_generation(gen, mating_size, offspring_size,retain_size):
    parents = selection(gen, mating_size)
    offsprings = create_offsprings(parents, offspring_size)
    return retain_best(parents+offsprings,retain_size)

In [75]:
# Generate a starting population (current) and iterate through gen_iters generations
def genetic_algorithm(gen_size, mating_size, offspring_size, retain_size, gens_count):
    current = starting_population(gen_size)
    for i in range(gens_count):
      current = new_generation(current, mating_size, offspring_size, retain_size)
    return current



In [78]:
def solve_TSP():
   best = max(genetic_algorithm(100,60,150,70,2000),key=lambda x: fitness(x))
   return (best, fitness(best))

In [79]:
print(solve_TSP())

(['Exet', 'Brig', 'Newc', 'Live', 'Oxfo', 'Glas', 'Edin', 'Nott', 'Aber', 'Inve', 'Lond', 'Stra'], 1733)
